# Temperature-Dependent EIS Analysis Part 1: Pre-RelaxIS Processing

As of February 2026, this is the most up-to-date EIS analysis and processing code that we have for the Maughan lab. This is the first installation of the analysis code where you will assign temperatures to .DTA files and identify which files you want to load into RelaxIS to fit.

In [1]:
import pandas as pd
import os
from datetime import datetime, timedelta, timezone
from PyEIS import *
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib.colors as mcolors
from matplotlib.pyplot import rc
import matplotlib.ticker as ticker
import csv
import glob
import statistics
from sklearn.linear_model import LinearRegression
from scipy.stats import linregress
import math
from math import pi
import re
from collections import defaultdict
import random
from scipy import stats





rc('text',usetex=True)
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']},size='16')
rc('text.latex', preamble=r'\usepackage{sfmath}')

In the cell below, you will specify:

1. What type of temperature log data you have (Quincy, manual-entry Excel file, or Thermotron)
2. Where your raw .DTA files live.
3. The parent folder where you will copy the spectra you will fit in RelaxIS.
4. Sample name and material name (use Latex formatting for material name)
5. Diameter and height (as well as associated error) of your pellet in mm

In [2]:
Quincy = False
Excel = True
Thermotron_log = False

temp_data_path =  '/Users/shelbygalinat/Documents/Documents/School/Grad_School/Research/EIS/SLG_3_023_Li6PS5OCN/SLG_3_023_Li6PS5OCN_temp_tracking.xlsx'
#if Quincy data, this is the folder in which quincy wrote all of his records. It will have a f*ckton of nested folders in it.
#otherwise, this will be the excel file or thermotron log file path

#this is the folder in which all of your .DTA files are for a given experiment
EIS_folder_path="/Users/shelbygalinat/Documents/Documents/School/Grad_School/Research/EIS/SLG_3_023_Li6PS5OCN/SLG_3_023_Li6PS5OCN/SLG_3_023_Li6PS5OCN_1"

#This is the folder in which your exported data will be saved
save_folder="/Users/shelbygalinat/Documents/Documents/School/Grad_School/Research/EIS/SLG_3_023_Li6PS5OCN/SLG_3_023_Li6PS5OCN/SLG_3_023_Li6PS5OCN_1/for_RelaxIS"

#INPUT DIAMETER AND HEIGHT OF PELLET HERE!
#diameter in mm
diameter = 6
#pellet height in mm
height = 1.112

#samplename and materialname are included in output files, so change them as appropriate, used in generating figure titles and metadata
samplename='SLG_3_023_Li6PS5OCN_1'
materialname='Li$_{6}$PS$_{5}$OCN'

#Errors in the thickness and height, generally no need to change this.
#err in pellet diameter (mm)
diameter_err=0.01
#err in pellet height (mm)
height_err=0.001

A gigantic function block, good luck soldier

In [ ]:
#Austin's function
#This function reads a given EIS dataset and returns the associated datetime
def extract_EIS_time(file_path):
    # Open the file in read mode
    with open(file_path, 'r', encoding='latin-1') as file:
        # Read the entire file content
        lines = file.readlines()

        # Look for the lines containing the date and time using regex
        #This reads the resulting file string for a string LABEL + tab + 2 digits / 2 digits / 4 digits
        date_regex = re.compile(r'DATE\tLABEL\t(\d{1,2}/\d{1,2}/\d{4})\tDate')
        time_regex = re.compile(r'TIME\tLABEL\t(\d{1,2}:\d{2}:\d{2})\tTime')
        date = None
        time = None

        for line in lines:
            # Check if the line matches the date pattern
            if date_regex.search(line):
                date = date_regex.search(line).group(1) #this takes the first string found
                #print(date)
            # Check if the line matches the time pattern
            if time_regex.search(line):
                time = time_regex.search(line).group(1)
                #print(time)
    #puts date and time in a single string
    date_time_str = date+'-'+time
    # Define the format in which the date and time are provided
    date_time_format = '%m/%d/%Y-%H:%M:%S'
    # Convert the string to a datetime object
    dt_object = datetime.datetime.strptime(date_time_str, date_time_format)
        
    return dt_object

#Austin's function
#Generates a dictionary of filename lists, where the key is the temperature and the list is the .dta files that are from the hold at that temperature
#Will throw an error for every empty .DTA file-- don't worry about that, just check that the first one is the first empty dta file
#Built so that it will do this process in numerical order !!!! We will need to adjust this function when QUINCY gets EIS capabilities again
def associate_EIS_with_temperature(EIS_folder_path, ovendata, printerrors=True):
    # Helper function to extract number from filename
    def extract_number(filename):
        match = re.search(r'\d+', filename)
        return int(match.group()) if match else float('inf')  # Send non-numeric to end

    # Get and sort .DTA files by number
    dta_files = sorted(
        [f for f in os.listdir(EIS_folder_path) if f.endswith('.DTA')],
        key=extract_number
    )

    # Dict to hold files grouped by temperature
    temp_to_files = defaultdict(list)
    
    # Loop through numerically sorted files
    for filename in dta_files:
        file_path = os.path.join(EIS_folder_path, filename)
        try:
            file_time = extract_EIS_time(file_path)
        except Exception as e:
            if printerrors:
                print(f"Could not read time from {filename} because it is likely empty: {e}")
            continue

        # Match file time to temperature hold window
        for _, row in ovendata.iterrows():
            if row['start_time'] <= file_time <= row['end_time']:
                temp_to_files[row['setpoint_C']].append(os.path.basename(file_path))
                break

    # Optional: sort dictionary by temperature
    temp_to_files_sorted = dict(sorted(temp_to_files.items()))
    return temp_to_files_sorted

#Austin's function
#function to generate a list of colors in hex format using start color, end color, and the target folder(directory) length
def color_gradient(startcolor,endcolor,temperatures):
    color_list = list(startcolor.range_to(endcolor, len(temperatures)))
    datacolor = {}
    for temp, color in zip(sorted(temperatures), color_list):
        datacolor[temp] = color.hex
    return datacolor

#Austin's function
# Define a function to format the axis multiplier
def format_axis(value, tick_number, limit):
    if limit >= 5e9:
        value = value / 1e9
    elif limit >= 5e6:
        value = value / 1e6
    elif limit >= 5e3:
        value = value / 1e3
    return '{:.0f}'.format(value)

#Austin's function
# Define a function to format the axis label unit
def labelprefix(limit):
    prefix = ''
    if limit >= 5e9:
        prefix = 'G'
    elif limit >= 5e6:
        prefix = 'M'
    elif limit >= 5e3:
        prefix = 'k'
    return prefix

#Austin's function
def generate_normalization_constant(diameter,height):
    area=pi*(((diameter/10)/2)**2)
    heightcm=height/10
    normconstant=area/heightcm
    return normconstant

#Austin's function
def get_round_locator_spacing(limit):
    # Choose a "nice" base spacing based on the axis limit
    scaler=1
    if limit >= 5e9:
        scaler=1e9
    elif limit >= 5e6:
        scaler= 1e6
    elif limit >= 5e3:
        scaler=1e3
    if limit > 5000*scaler:
        return 1000*scaler
    elif limit > 2000*scaler:
        return 500*scaler
    elif limit > 1000*scaler:
        return 250*scaler
    elif limit > 500*scaler:
        return 100*scaler
    elif limit > 200*scaler:
        return 50*scaler
    elif limit > 100*scaler:
        return 20*scaler
    elif limit > 50*scaler:
        return 10*scaler
    elif limit > 20*scaler:
        return 5*scaler
    elif limit > 10*scaler:
        return 2*scaler
    else:
        return 1*scaler

'''Shelby's function to convert an excel file used for manual temperature tracking to a temperature/time dataframe
This assumes the excel file has been organized in a specific format - see Shelby's example file
'''
def extract_manual_excel_temp_data(file_path):
    # Read the raw excel file with the Thermotron program info
    raw_df = pd.read_excel(file_path,header=0,skiprows=3)
    df = raw_df[raw_df['time (hr:min)'] != datetime.time(0, 0)].copy() #save a new dataframe with only non-zero times, since these are the ramp intervals as opposed to the hold intervals
    df['setpoint_C']=df['IV (˚C)'] #add a column for setpoint to match Austin's previous dataframe for Quincy 
    df=df.drop(columns=['time (hr:min)','IV (˚C)','FV (˚C)']) #drop these columns since they aren't needed
    df.dropna(how='all', axis=1, inplace=True) #drop any columns that are completely empty
    return df


#function below copied from Austin's old code
#This function finds all folders labelled "Hold" in the quincy file sorting system and reads the metadata to find the start time, end time, and temperature
def extract_hold_metadata(parent_path):
    hold_data = []

    # Walk through all directories and subdirectories
    for root, dirs, files in os.walk(parent_path):
        # Check if this folder contains 'Hold' in its name
        if "Hold" in os.path.basename(root):
            metadata_path = os.path.join(root, 'metadata.csv')
            if os.path.isfile(metadata_path):
                try:
                    # Read the metadata.csv file
                    df = pd.read_csv(metadata_path)

                    # Extract the first row
                    row = df.iloc[0]
                    
                    start_time = datetime.datetime.strptime(row.iloc[0], '%d %B %Y %I:%M:%S %p')
                    end_time = datetime.datetime.strptime(row.iloc[1], '%d %B %Y %I:%M:%S %p')
                    setpoint = float(row.iloc[3])

                    hold_data.append({
                        'folder': root,
                        'start_time': start_time,
                        'end_time': end_time,
                        'setpoint_C': setpoint
                    })
                except Exception as e:
                    print(f"Failed to read {metadata_path}: {e}")

    return pd.DataFrame(hold_data)

#Shelby/ChatGPT's code for finding stable regions in thermotron temp log file and recording those 
def extract_thermo_temp_data(file_path):
    variance=1.0,
    min_points=3   # require stability for at least N timestamps

    df = pd.read_csv(file_path)

    # Parse + clean
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['temperature_C'] = pd.to_numeric(df['temperature_C'])
    df['setpoint_C'] = pd.to_numeric(df['setpoint_C'])

    # Check if temperature is within tolerance. df['stable'] is a Boolean
    df['stable'] = (
        (df['temperature_C'] >= df['setpoint_C'] - variance) &
        (df['temperature_C'] <= df['setpoint_C'] + variance)
    )

    # Identify contiguous regions where:
    # 1) stability state changes OR
    # 2) setpoint changes
    df['block'] = (
        (df['stable'] != df['stable'].shift()) | #compares value of stability Boolean in current row and the row before 
        (df['setpoint_C'] != df['setpoint_C'].shift()) #compares value of setpoint in current row and the row before
    ).cumsum() #calculate a numerical index that increases with cumulative sum for the block number 

    results = []

    for _, block in df.groupby('block'): #iterate over the block column in the dataframe
        if not block['stable'].iloc[0]: #don't count any blocks that are unstable
            continue

        if len(block) < min_points: #don't count any blocks that are stable for less than 3 timestamps at 30 second intervals, so 1.5 min
            continue

        results.append({ #add a dictionary for each block
            'start_time': block['timestamp'].iloc[0], #timestamp in this row is the start time
            'end_time': block['timestamp'].iloc[-1], #timestamp in the previous row is the end time... is this right?
            'setpoint_C': block['setpoint_C'].iloc[0]
        })
        
    result_df = pd.DataFrame(results)
   # keep only the LAST stable block per setpoint in case there are multiple periods of stability at one setpoint
    result_df = (
        result_df
        .sort_values('start_time')
        .groupby('setpoint_C', as_index=False)
        .tail(1) #keeps only the last block at this setpoint
        .reset_index(drop=True)
    )


    return result_df


#Shelby's function to determine which style of temperature data to import
def extract_temp_data(file_path):
    if Quincy:
        df = extract_hold_metadata(file_path)
    if Excel:
        df = extract_manual_excel_temp_data(file_path)
    if Thermotron_log:
        df = extract_thermo_temp_data(file_path)
    return df

#Shelby's function to generate dataframes from all of the .DTA files and plot the Nyquists for each temperature, partially adapted from
#truncated ausaplot in Austin's old code
#plotall is a boolean that if true, will plot all of the temperatures in the nyquist dictionary
#selected_temperatures is which temperatures you want to plot. It is overridden if plotall=True
#keep_last_n: keep the last n replicates at each temperature
#returndatasets is a Boolean that if true, will return the selected dataframes for future use
def visualize_nyquists(files_found_dict,plotall,selected_temperatures,start_color,end_color,keep_last_n=None,returndatasets=True):
    nyquist_dict = {} #create a dictionary to store the nyquists
    # Iterate through the temperature keys in files_found_dict
    for temp, file_list in files_found_dict.items():
        for idx, filename in enumerate(file_list, start=1):
            fullpath = os.path.join(EIS_folder_path, filename)
            df = pd.read_csv(fullpath, skiprows=52, delimiter='\t',encoding='latin1',header=0) # Read the file, skipping the first 52 rows to get to the header 
            df = df.drop([0]) # Drop the rows that contain the units (index 0) 
            df = df.reset_index(drop=True) # Reset the index 
            df=df.apply(pd.to_numeric, errors="coerce") #change all the datatypes to floats - for some reason they were being read as strings
            nyquist_dict[temp,idx,filename]=df #add the dataframe to the nyquist dictionary
        
    # Create colormap
    cmap = mcolors.LinearSegmentedColormap.from_list("gradient", [start_color, end_color])
    
    #key[0] is the temperature, key[1] is the number of replicates for a specific temp
    # Normalize the number of replicates globally (across all temperatures) for color gradient purposes, I think
    all_keys = [key[0] for key in nyquist_dict.keys()]
    norm = mcolors.Normalize(vmin=min(all_keys), vmax=max(all_keys))
    
    # Group DataFrames by key[0], which is the temperature
    grouped_nyquist = defaultdict(list)
    for key, df in nyquist_dict.items():
        grouped_nyquist[key[0]].append((key, df))
        
    # Sort each group's dataframes by key[1], which is the replicate number 
    for group_key in grouped_nyquist:
        grouped_nyquist[group_key].sort(key=lambda x: x[0][1])
        
    for temp in grouped_nyquist:
        datasets = grouped_nyquist[temp]
        # Apply truncation
        if keep_last_n is not None:
            datasets = datasets[-keep_last_n:]
        grouped_nyquist[temp] = datasets
    # Determine temps to plot
    if plotall:
        temps_to_plot = sorted(grouped_nyquist.keys())

    else:
        temps_to_plot = []
        for temp in selected_temperatures:
            if temp not in grouped_nyquist:
                print(f"Warning: Temperature {temp}°C not found in data. Skipping.")
            else:
                temps_to_plot.append(temp)

    if not temps_to_plot:
        raise ValueError("No valid temperatures found to plot.")

    n = len(temps_to_plot)
    cols = math.ceil(math.sqrt(n))
    rows = math.ceil(n / cols)

    width_per_plot = 6
    height_per_plot = 6
    fig, axes = plt.subplots(rows, cols, figsize=(cols * width_per_plot, rows * height_per_plot), squeeze=False)

    for ax in axes.flatten()[n:]:
        ax.set_visible(False)

    for temp_index,(ax, temperature) in enumerate(zip(axes.flatten(), temps_to_plot)):
        dataframes = grouped_nyquist[temperature]
        reasonable_max_Zreals_for_plot = []
        for i, (key, df_original) in enumerate(dataframes):
            df=df_original.copy()
            color = "#ff66ff" if i == len(dataframes) - 1 else cmap(norm(key[1]))
            ax.plot(df['Zreal'], -df['Zimag'], linestyle='none', marker='o', color=color, markerfacecolor='white')
            reasonable_max_Zreal_for_plot = df['Zreal'][len(df['Zreal'])-1-temp_index*5] #Shelby's jank code for getting a reasonable x limit, assuming you want to zoom in more at higher temps
            reasonable_max_Zreals_for_plot.append(reasonable_max_Zreal_for_plot)
        max_reasonable_max_Zreal_for_plot = max(reasonable_max_Zreals_for_plot)
        limit = max_reasonable_max_Zreal_for_plot
        ax.set(xlim=(0, limit), ylim=(0, limit))

        prefix = labelprefix(limit)

        def custom_formatter(limit):
            def formatter(x, pos):
                return format_axis(x, pos, limit)
            return formatter

        ax.xaxis.set_major_formatter(ticker.FuncFormatter(custom_formatter(limit)))
        ax.yaxis.set_major_formatter(ticker.FuncFormatter(custom_formatter(limit)))

        tick_spacing=get_round_locator_spacing(limit)
        locator = ticker.MultipleLocator(tick_spacing)
        ax.xaxis.set_major_locator(locator)
        ax.yaxis.set_major_locator(locator)

        ax.set_xlabel(rf'Z$_{{real}}$ [{prefix}$\Omega$]')
        ax.set_ylabel(rf'-Z$_{{imag}}$ [{prefix}$\Omega$]')

        for spine in ax.spines.values():
            spine.set_linewidth(2)
        ax.yaxis.set_minor_locator(ticker.AutoMinorLocator(2))
        ax.xaxis.set_minor_locator(ticker.AutoMinorLocator(2))
        ax.tick_params(which='both', direction='in', right=True, top=True)
        ax.tick_params(which='both', direction='in',length=6, right=False, top=False)
        ax.tick_params(which='minor', direction='in',length=3, right=False, top=False)
        ax.tick_params(width=2,which='both')

    if n == 1:
        axes.flatten()[0].set_title(rf'{materialname} {temperature} $^\circ$C')
    else:
        fig.suptitle(materialname, y=0.99, x=.52)
        for ax, temperature in zip(axes.flatten(), temps_to_plot):
            ax.set_title(rf'{temperature} $^\circ$C')
    plt.tight_layout()
    return(grouped_nyquist)
    plt.show()

#Shelby's function for copying the original .DTA files into a new directory with more useful filenames and with 
#metadata for RelaxIS appended to the start
def copy_DTAs_with_metadata(grouped_nyquist_dict,save_folder):
    for temp_index,(temperature) in enumerate(sorted(grouped_nyquist_dict.keys())): 
        dataframes = grouped_nyquist[temperature]
        for i, (key, df) in enumerate(dataframes,start=1):
            old_filename_as_DTA = key[2]
            #old_filename_as_txt = key[2][:len(key[2])-4]+".txt"#save as .txt instead of .DTA, so that RelaxIS will read in the file as an unknown dataset so you can set the metadata you want
            new_filename=str(key[0])+"C_replicate_"+str(i)+"_"+old_filename_as_DTA
            #new_filename=str(key[0])+"C_replicate_"+str(i)+"_"+old_filename_as_txt #generate a new .txt filename that appends 
            #temp and replicate number to the first part of the original filename
            old_filepath = os.path.join(EIS_folder_path, f"{key[2]}")
            save_filepath = os.path.join(save_folder, f"{new_filename}")
            new_lines = "Temperature: "+str(key[0])+"C \nReplicate: "+str(i)+"\nThickness: "+str(round(height/10,4))+" cm\nArea: "+str(round(np.pi*(diameter/10/2)**2,2))+" cm^2\n" #add lines to the .DTA file that include temp, replicate number, thickness (cm), and area (cm^2)
            with open(old_filepath, "r", encoding="latin-1") as fin, \
                 open(save_filepath, "w", encoding="latin-1") as fout:
                fout.write(new_lines) 
                for line in fin:
                    fout.write(line) 



Start running the code

In [ ]:
#generate the temperature dataframe to feed into the next function
rawovendata=extract_temp_data(temp_data_path)

#make a dictionary of lists of the .DTA files. The key for each list is the temperature at which they were collected
#each list is ordered from lowest X to highest X in EISPOT_#X 
files_found_dict=associate_EIS_with_temperature(EIS_folder_path=EIS_folder_path,ovendata=rawovendata,printerrors=False)

#quickly visualize the raw Nyquists at each temperature and determine how many replicates at each temperature
#you would like to fit in RelaxIS
grouped_nyquist={}
grouped_nyquist=visualize_nyquists(files_found_dict,plotall=True,selected_temperatures=[],start_color="#3399ff", end_color="#ff66ff",keep_last_n=3)

#copy the DTA files for the selected final replicates at each temperature with additional metadata appended to the start into the save folder for fitting in RelaxIS
copy_DTAs_with_metadata(grouped_nyquist,save_folder)